### Week-5 Assignment

#### Most neighbourhoods around Toronto have more recreational venues like restaurants, bars, theatres etc..and do not have many pharmacies and hospitals. Therefore the business problem that is addressed as part of this project, is to identify potential neighbourhoods to open a pharmacyin the interest of the public.

##### Import libraries

In [1]:
# Import all required libraries

from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests

url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

page =requests.get(url).text

soup=BeautifulSoup(page,'html5lib')

##### Scrap data from Wikipedia for neighboiurhoods 

In [2]:
# Webscraping as per hint given

table_contents=[]
table = soup.find('table')
for row in table.find_all('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [3]:
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


##### Import data from csv file for geospatial data and filter dataset for Toronto Neighbourhoods only

In [3]:
#Read the geospatial csv file
data_geospatial=pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv')

#renaming the postalcode column to be same as the earlier dataframe
data_geospatial.rename(columns={'Postal Code':'PostalCode'},inplace=True)
data_geospatial.head()


data_all=pd.merge(df,data_geospatial,on='PostalCode')
data_all.head(12)


Toronto_data=data_all[data_all["Borough"].str.contains('Toronto')].reset_index(drop=True)
Toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


##### Install folium for maps

In [4]:
!pip install folium

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 94 kB 6.1 MB/s  eta 0:00:01


##### Get location data for Toronto

In [5]:
import folium
from geopy.geocoders import Nominatim 

address = 'Toronto, Ontario Canada'
geolocator = Nominatim(user_agent="Foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [7]:
Toronto_map = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, label in zip(Toronto_data['Latitude'], Toronto_data['Longitude'], Toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat, lng], radius=5, popup=label, color='blue', fill=True, fill_color='#3186cc', fill_opacity=0.7,parse_html=False).add_to(Toronto_map)  
#map_toronto

##### Get Foursquare credentials

In [6]:
CLIENT_ID = 'NMXVKO4C3USUDS2PBK4UTU2THFYJZ44QHIYVZETTK1HAVOMR' # your Foursquare ID
CLIENT_SECRET = 'KKJVDSFXIUCBQDMXYEKONGNE501JQHALG32EGPGVPVRRSAQM' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: NMXVKO4C3USUDS2PBK4UTU2THFYJZ44QHIYVZETTK1HAVOMR
CLIENT_SECRET:KKJVDSFXIUCBQDMXYEKONGNE501JQHALG32EGPGVPVRRSAQM


##### Get venues for all Neighbourhoods

In [7]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [8]:
Toronto_venues = getNearbyVenues(names=Toronto_data['Neighborhood'],
                                   latitudes=Toronto_data['Latitude'],
                                   longitudes=Toronto_data['Longitude']
                                  )

Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Enclave of M5E
St. James Town, Cabbagetown
First Canadi

In [30]:
#print(Toronto_venues.shape)
#Toronto_venues.head()
n=len(pd.unique(Toronto_venues['Neighborhood']))
print(n)

39


In [9]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhoods'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

print(Toronto_onehot.shape)
Toronto_onehot.head()

(1599, 231)


,Neighborhoods,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
Toronto_grouped = Toronto_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(Toronto_grouped.shape)
Toronto_grouped

(39, 231)


,Neighborhoods,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.00000,0.000000,0.00000,0.00,0.017544,0.000000,0.000000,0.000000,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"CN Tower, King and Spadina, Railway Lands, Har...",0.055556,0.055556,0.055556,0.111111,0.166667,0.111111,0.000000,0.000000,0.00,...,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.00000,0.000000,0.00000,0.00,0.014706,0.000000,0.000000,0.014706,0.000000,0.014706
4,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,Church and Wellesley,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012658,0.000000,0.00,...,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.025316
6,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.00,...,0.00000,0.000000,0.00000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000
7,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.00000,0.030303,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,"Dufferin, Dovercourt Village",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [11]:
Toronto_pharmacy = Toronto_grouped[["Neighborhoods","Pharmacy"]]
Toronto_pharmacy.head()

,Neighborhoods,Pharmacy
0,Berczy Park,0.035088
1,"Brockton, Parkdale Village, Exhibition Place",0.000000
2,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000
3,Central Bay Street,0.000000
4,Christie,0.000000


In [12]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

kl_clustering = Toronto_pharmacy.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]


array([2, 0, 0, 0, 0, 0, 0, 2, 0, 1], dtype=int32)

In [13]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
Toronto_merged = Toronto_pharmacy.copy()

# add clustering labels
Toronto_merged["Cluster Labels"] = kmeans.labels_


In [14]:
Toronto_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
Toronto_merged.head()

,Neighborhood,Pharmacy,Cluster Labels
0,Berczy Park,0.035088,2
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0
2,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0
3,Central Bay Street,0.000000,0
4,Christie,0.000000,0


In [15]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(Toronto_data.set_index("Neighborhood"), on="Neighborhood")

print(Toronto_merged.shape)
Toronto_merged.head() # check the last columns!

(39, 7)


,Neighborhood,Pharmacy,Cluster Labels,PostalCode,Borough,Latitude,Longitude
0,Berczy Park,0.035088,2,M5E,Downtown Toronto,43.644771,-79.373306
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0,M6K,West Toronto,43.636847,-79.428191
2,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0,M5V,Downtown Toronto,43.628947,-79.394420
3,Central Bay Street,0.000000,0,M5G,Downtown Toronto,43.657952,-79.387383
4,Christie,0.000000,0,M6G,Downtown Toronto,43.669542,-79.422564


In [47]:
# sort the results by Cluster Labels
print(Toronto_merged.shape)
Toronto_merged.sort_values(["Cluster Labels"], inplace=True)
Toronto_merged

(39, 7)


,Neighborhood,Pharmacy,Cluster Labels,PostalCode,Borough,Latitude,Longitude
19,Lawrence Park,0.000000,0,M4N,Central Toronto,43.728020,-79.388790
20,"Little Portugal, Trinity",0.000000,0,M6J,West Toronto,43.647927,-79.419750
21,"Moore Park, Summerhill East",0.000000,0,M4T,Central Toronto,43.689574,-79.383160
22,North Toronto West,0.000000,0,M4R,Central Toronto,43.715383,-79.405678
23,"Parkdale, Roncesvalles",0.000000,0,M6R,West Toronto,43.648960,-79.456325
24,"Regent Park, Harbourfront",0.000000,0,M5A,Downtown Toronto,43.654260,-79.360636
25,"Richmond, Adelaide, King",0.000000,0,M5H,Downtown Toronto,43.650571,-79.384568
37,"Toronto Dominion Centre, Design Exchange",0.000000,0,M5K,Downtown Toronto,43.647177,-79.381576
26,Rosedale,0.000000,0,M4W,Downtown Toronto,43.679563,-79.377529
28,"Runnymede, Swansea",0.000000,0,M6S,West Toronto,43.651571,-79.484450


In [16]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [36]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0]


,Neighborhood,Pharmacy,Cluster Labels,PostalCode,Borough,Latitude,Longitude
1,"Brockton, Parkdale Village, Exhibition Place",0.0,0,M6K,West Toronto,43.636847,-79.428191
2,"CN Tower, King and Spadina, Railway Lands, Har...",0.0,0,M5V,Downtown Toronto,43.628947,-79.394420
3,Central Bay Street,0.0,0,M5G,Downtown Toronto,43.657952,-79.387383
4,Christie,0.0,0,M6G,Downtown Toronto,43.669542,-79.422564
5,Church and Wellesley,0.0,0,M4Y,Downtown Toronto,43.665860,-79.383160
6,"Commerce Court, Victoria Hotel",0.0,0,M5L,Downtown Toronto,43.648198,-79.379817
8,Davisville North,0.0,0,M4P,Central Toronto,43.712751,-79.390197
10,Enclave of M4L,0.0,0,M7Y,East Toronto Business,43.662744,-79.321558
12,"First Canadian Place, Underground city",0.0,0,M5X,Downtown Toronto,43.648429,-79.382280
13,Forest Hill North & West,0.0,0,M5P,Central Toronto,43.696948,-79.411307


In [18]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1]

,Neighborhood,Pharmacy,Cluster Labels,PostalCode,Borough,Latitude,Longitude
9,"Dufferin, Dovercourt Village",0.142857,1,M6H,West Toronto,43.669005,-79.442259


In [19]:
        Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2]

,Neighborhood,Pharmacy,Cluster Labels,PostalCode,Borough,Latitude,Longitude
0,Berczy Park,0.035088,2,M5E,Downtown Toronto,43.644771,-79.373306
7,Davisville,0.030303,2,M4S,Central Toronto,43.704324,-79.388790


In [20]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3]

,Neighborhood,Pharmacy,Cluster Labels,PostalCode,Borough,Latitude,Longitude
11,Enclave of M5E,0.020408,3,M5W,Downtown Toronto Stn A,43.646435,-79.374846
18,"Kensington Market, Chinatown, Grange Park",0.014493,3,M5T,Downtown Toronto,43.653206,-79.400049
29,St. James Town,0.012346,3,M5C,Downtown Toronto,43.651494,-79.375418
30,"St. James Town, Cabbagetown",0.023256,3,M4X,Downtown Toronto,43.667967,-79.367675


In [21]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4]

,Neighborhood,Pharmacy,Cluster Labels,PostalCode,Borough,Latitude,Longitude
33,"The Annex, North Midtown, Yorkville",0.047619,4,M5R,Central Toronto,43.67271,-79.405678
